In [1]:
import pandas as pd
from scipy.stats import poisson

In [15]:
data = pd.read_csv("data.csv", index_col=0).sort_values(by='date')
split_ratio = 0.8  
split_index = int(len(data) * split_ratio)

# Split the DataFrame
train_df = data.iloc[:split_index]
val_df = data.iloc[split_index:]

In [17]:
mean = train_df["points"].mean()
print(mean)


26.057142857142857


In [20]:
lambda_ = mean
n = 29
probability = 1 - poisson.cdf(n, lambda_)
print(probability)

0.24432256265010233


In [21]:
success = val_df[val_df['points'] >= n]
failure = val_df[val_df['points'] < n]
print(len(success)/len(val_df))

0.4444444444444444
